In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LassoCV
from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.metrics import make_scorer, mean_squared_error

In [2]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [3]:
def load_data():
    train = pd.read_csv('../input/train.csv')
    test = pd.read_csv('../input/test.csv')
    combined = train.append(test, ignore_index=True).drop(['Id','SalePrice'], axis=1)
    combined = pd.get_dummies(combined)
    combined = combined.fillna(combined.mean())
    y = np.log(train['SalePrice'].values)
    X = combined.iloc[:train.shape[0],:]
    X_submission = combined.iloc[train.shape[0]:,:]
    ids_submission = test['Id'].values
    return y, X, X_submission, ids_submission

In [4]:
# load data
y, X, X_submission, ids_submission = load_data()

In [5]:
model = LassoCV(cv=KFold(10, random_state=1773), random_state=1773, normalize=True)

In [6]:
model.fit(X,y)

LassoCV(alphas=None, copy_X=True,
    cv=KFold(n_splits=10, random_state=1773, shuffle=False), eps=0.001,
    fit_intercept=True, max_iter=1000, n_alphas=100, n_jobs=1,
    normalize=True, positive=False, precompute='auto', random_state=1773,
    selection='cyclic', tol=0.0001, verbose=False)

In [7]:
model.alpha_

0.00021154302860138105

In [8]:
print 'RMSE:', np.min(np.sqrt(model.mse_path_.mean(axis=-1)))

RMSE: 0.145816619925


In [9]:
coefs = pd.DataFrame({'variable': X.columns, 'coef': model.coef_})